# 0) Setup and load data

In [8]:
import os
import pickle
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier

DATA_PATH = os.path.join("..", "data", "hotel_bookings.csv")

df = pd.read_csv(DATA_PATH)
df.shape, df.columns[:10]

((119390, 32),
 Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
        'arrival_date_month', 'arrival_date_week_number',
        'arrival_date_day_of_month', 'stays_in_weekend_nights',
        'stays_in_week_nights', 'adults'],
       dtype='object'))

# 1) Feature engineering

In [9]:
df_model = df.copy()

#basic target
y = df_model["is_canceled"].astype(float)

#simple, stable features (mostly numeric)
df_model["total_guests"] = (
    df_model.get("adults", 0).fillna(0)
    + df_model.get("children", 0).fillna(0)
    + df_model.get("babies", 0).fillna(0)
)

#binary encode hotel type (City=1, Resort=0)
df_model["hotel_city"] = (df_model["hotel"] == "City Hotel").astype(int)
feature_cols = [
    "lead_time",
    "adr",
    "stays_in_week_nights",
    "stays_in_weekend_nights",
    "total_guests",
    "hotel_city",
    "previous_cancellations",
    "booking_changes"
]

X_df = df_model[feature_cols].copy()

#quickly hande missing values
X_df = X_df.replace([np.inf, -np.inf], np.nan)
mask = X_df.notna().all(axis=1) & y.notna()

X = X_df.loc[mask].to_numpy(dtype=float)
y = y.loc[mask].to_numpy(dtype=float)

X.shape, y.shape

((119390, 8), (119390,))

# 2) Train/test split

In [10]:
#1) reproducible shuffle
rng = np.random.default_rng(42)
n = X.shape[0]
idx = rng.permutation(n)

#2) train/test split (80/20)
split = int(0.8 * n)
train_idx = idx[:split]
test_idx = idx[split:]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test = X[test_idx], y[test_idx]

print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes:", X_test.shape, y_test.shape)

#3) z-score standardization
mu = X_train.mean(axis=0)
sigma = X_train.std(axis=0)
sigma[sigma == 0] = 1.0

X_train_s = (X_train - mu) / sigma
X_test_s = (X_test - mu) / sigma

#4) intercept (design matrix)
X_train_design = np.column_stack([np.ones(X_train_s.shape[0]), X_train_s])
X_test_design = np.column_stack([np.ones(X_test_s.shape[0]), X_test_s])

print("Design shapes:", X_train_design.shape, X_test_design.shape)
print("First row (train):", X_train_design[0])



Train shapes: (95512, 8) (95512,)
Test shapes: (23878, 8) (23878,)
Design shapes: (95512, 9) (23878, 9)
First row (train): [ 1.         -0.76832031  0.72694085  0.26633103  0.07679599  0.04223071
  0.70918444 -0.10225844 -0.33922077]


# 3) Fit a NumPy Linear Model

In [11]:
#fit model using least squares: minimize squared errors
beta, residuals, ran, s = np.linalg.lstsq(X_train_design, y_train, rcond=None)

#predictions on train and test
y_pred_train = X_train_design @ beta
y_pred_test = X_test_design @ beta

#quick sanity checks (first 5 predictions)
print("beta shape:", beta.shape)
print("First 5 train predictions:", y_pred_train[:5])
print("First 5 test predictions: ", y_pred_test[:5])

#clip predictions to [0,1] for interpretability
y_pred_test_clipped = np.clip(y_pred_test, 0, 1)
print("Test predictions clipped range:", y_pred_test_clipped.min(), y_pred_test_clipped.max())


beta shape: (9,)
First 5 train predictions: [0.34251092 0.31006186 0.15642004 0.44584317 0.09932604]
First 5 test predictions:  [0.33954775 0.83666817 0.52176555 0.17107856 0.60683405]
Test predictions clipped range: 0.0 1.0


In [35]:
#evaluation metrics
def mse(y_true, y_pred):                
    return np.mean((y_true - y_pred) ** 2)

def r2(y_true, y_pred):             #how much of the variance is explained by the model
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - ss_res / ss_tot if ss_tot != 0 else np.nan

#model performance
train_mse = mse(y_train, y_pred_train)          #how the model adapts to training data
test_mse = mse(y_test, y_pred_test)             #how good the model generalizes

train_r2 = r2(y_train, y_pred_train)
test_r2 = r2(y_test, y_pred_test)

print("Model performance:")
print(f"Train MSE: {train_mse:.4f}")
print(f"Test  MSE: {test_mse:.4f}")
print(f"Train R²:  {train_r2:.4f}")
print(f"Test  R²:  {test_r2:.4f}")

#baseline model
baseline_pred_train = np.full_like(y_train, y_train.mean())             #use a naive model, always predicts the same value
baseline_pred_test = np.full_like(y_test, y_train.mean())

baseline_train_mse = mse(y_train, baseline_pred_train)              #check how good is the model compared to baseline
baseline_test_mse = mse(y_test, baseline_pred_test)


print("\nBaseline performance:")
print(f"Baseline Train MSE: {baseline_train_mse:.4f}")
print(f"Baseline Test  MSE: {baseline_test_mse:.4f}")

Model performance:
Train MSE: 0.2030
Test  MSE: 0.2038
Train R²:  0.1297
Test  R²:  0.1261

Baseline performance:
Baseline Train MSE: 0.2332
Baseline Test  MSE: 0.2332


**Linear Probability Model Interpretation**

The linear probability model serves as a simple and interpretable baseline for analyzing cancellation behavior. It outperforms a naive mean-based baseline, indicating that booking characteristics contain useful predictive information, although overall predictive performance remains limited due to the binary nature of the target. Consequently, the model is primarily valuable for understanding the direction and relative importance of key features rather than for accurate individual-level prediction.

# 4) Logistic Regression for Cancellation Prediction

To complement the linear probability model, we fit a logistic regression model to predict booking cancellations. Logistic regression is more appropriate for binary outcomes, as it directly models probabilities in the [0,1] interval and often provides improved predictive performance. The goal of this section is to assess whether a more flexible model can outperform the linear model in terms of prediction accuracy.

In [12]:
#Fit logistic regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_s, y_train)                  #learn connection between features and cancellation probability using sigmoid function

#Predictions
y_pred_test_lr = logreg.predict(X_test_s)               #retuns class probabilities (0 or 1)
y_prob_test_lr = logreg.predict_proba(X_test_s)[:, 1]       #returns predicted cancellation probabilities

#Evaluation metrics
acc = accuracy_score(y_test, y_pred_test_lr)
auc = roc_auc_score(y_test, y_pred_test_lr)             #how good does the model separate between cancelled and not-cancelled bookings

print("Logistic Regression Performance (Test set):")
print(f"Accuracy: {acc:.3f}")
print(f"AUC:      {auc:.3f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_test_lr))


Logistic Regression Performance (Test set):
Accuracy: 0.700
AUC:      0.632

Classification Report:
              precision    recall  f1-score   support

         0.0       0.71      0.90      0.79     15036
         1.0       0.67      0.37      0.48      8842

    accuracy                           0.70     23878
   macro avg       0.69      0.63      0.63     23878
weighted avg       0.69      0.70      0.67     23878



**Logistic Regression Results Interpretation**

The logistic regression model achieves a test accuracy of 0.70 and an AUC of 0.632, indicating moderate predictive performance. The model correctly identifies most non-canceled bookings, but has lower recall for canceled bookings, reflecting the inherent difficulty of predicting cancellations. Overall, logistic regression provides improved and more realistic predictions compared to the linear probability model.

# 5) Random Forest Classifier

In [30]:
#Fit Random Forest
rf = RandomForestClassifier(
    n_estimators=300,                       #number of trees
    max_depth=15,                           #decreases overfitting
    min_samples_leaf=20,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_s, y_train)                

#Predictions
y_pred_test_rf = rf.predict(X_test_s)              
y_prob_test_rf = rf.predict_proba(X_test_s)[:, 1]       

#Evaluation
acc_rf = accuracy_score(y_test, y_pred_test_rf)
auc_rf = roc_auc_score(y_test, y_pred_test_rf)              

print("Random Forest Performance (Test set):")
print(f"Accuracy: {acc_rf:.3f}")
print(f"AUC:      {auc_rf:.3f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_test_rf))

# 4) Feature importances
importances = pd.Series(rf.feature_importances_, index=feature_cols).sort_values(ascending=False)           #how much every variable affects decrease in impurity
print("\nTop feature importances:")
print(importances.head(10))


Random Forest Performance (Test set):
Accuracy: 0.770
AUC:      0.711

Classification Report:
              precision    recall  f1-score   support

         0.0       0.76      0.94      0.84     15036
         1.0       0.82      0.48      0.61      8842

    accuracy                           0.77     23878
   macro avg       0.79      0.71      0.72     23878
weighted avg       0.78      0.77      0.75     23878


Top feature importances:
lead_time                  0.385237
previous_cancellations     0.187952
adr                        0.185512
booking_changes            0.099256
stays_in_week_nights       0.048424
hotel_city                 0.034591
total_guests               0.032751
stays_in_weekend_nights    0.026277
dtype: float64


The random forest model achieves the strongest predictive performance and improves recall for canceled bookings. Feature importance analysis consistently identifies lead time, previous cancellations, and ADR as the most important drivers of cancellations.

In [34]:
#tuning of max depth parameter
results = []

for depth in [8, 10, 12, 15]:
    rf = RandomForestClassifier(
        n_estimators=300,
        max_depth=depth,
        min_samples_leaf=20,
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train_s, y_train)
    y_prob = rf.predict_proba(X_test_s)[:, 1]
    auc = roc_auc_score(y_test, y_prob)
    results.append((depth, auc))

results


[(8, 0.7832574361681253),
 (10, 0.7963537927431528),
 (12, 0.8097957723600131),
 (15, 0.8252420083377967)]

A limited hyperparameter search over the maximum tree depth shows a consistent improvement in AUC as model complexity increases. The random forest with max_depth = 15 achieves the best performance, indicating that non-linear effects and feature interactions play an important role in cancellation prediction. No clear signs of overfitting are observed within the tested range.

# 6) Model evaluation and baseline performance

In [ ]:
comparison = pd.DataFrame({
    "Model": [
        "Linear Probability Model (NumPy)",
        "Logistic Regression",
        "Random Forest (tuned)"
    ],
    "Metrics": [
        "Test MSE = 0.2038, Test R² = 0.1261",     # <-- replace if you recomputed
        "Test Accuracy = 0.700, Test AUC = 0.632", # <-- your output
        "Test Accuracy = 0.770, Test AUC = 0.711"  # <-- your tuned RF output
    ],
    "Best For": [
        "Interpretability (direction of effects)",
        "Simple prediction + probability estimates",
        "Best predictive performance"
    ],
    "Key Notes": [
        "Outperforms baseline MSE (mean predictor)",
        "Struggles to recall cancellations (class 1)",
        "Improves recall for cancellations + captures non-linearity"
    ]
})

comparison
